In [1]:
# Libraries for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf

---

<h1><center>Data Pre-Processing

---

In [2]:
# Download data
df = pd.read_csv('ev_battery_charging_data.csv')

<h3><center>Data Cleaning

In [3]:
# Drop rows with any NaN values
df = df.dropna()

# Shape after cleaning
print("\nAfter removing rows with NaN:")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")


After removing rows with NaN:
Rows: 1000, Columns: 13


In [4]:
# Drop inputs calculated from the column we are predicting
drop_cols = ['Degradation Rate (%)', 'Efficiency (%)', 'Optimal Charging Duration Class']
df = df.drop(columns=drop_cols)

In [5]:
df.head()

,SOC (%),Voltage (V),Current (A),Battery Temp (°C),Ambient Temp (°C),Charging Duration (min),Charging Mode,Battery Type,Charging Cycles,EV Model
0,43.708611,3.629593,33.553512,33.454060,26.439918,59.363552,Fast,Li-ion,112,Model B
1,95.564288,3.879331,32.228092,35.933628,31.108647,67.343566,Fast,LiFePO4,398,Model A
2,75.879455,4.111062,91.562912,25.009358,30.203219,105.454739,Slow,LiFePO4,175,Model A
3,63.879264,4.012557,32.459158,32.497482,18.077998,54.000439,Fast,LiFePO4,150,Model B
4,24.041678,4.064593,34.475475,31.434920,17.984989,106.964968,Fast,Li-ion,886,Model C


<h3><center>One-Hot Encode Categorical Data

In [6]:
# One-hot encode charging mode, battery type, and EV model
df = pd.get_dummies(df, columns=['Charging Mode', 'Battery Type', 'EV Model'], prefix='', prefix_sep='', dtype=float)

In [7]:
# Move the column we are predicting to the end
df['Charging Duration (min)'] = df.pop('Charging Duration (min)')

In [8]:
df.head()

,SOC (%),Voltage (V),Current (A),Battery Temp (°C),Ambient Temp (°C),Charging Cycles,Fast,Normal,Slow,Li-ion,LiFePO4,Model A,Model B,Model C,Charging Duration (min)
0,43.708611,3.629593,33.553512,33.454060,26.439918,112,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,59.363552
1,95.564288,3.879331,32.228092,35.933628,31.108647,398,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,67.343566
2,75.879455,4.111062,91.562912,25.009358,30.203219,175,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,105.454739
3,63.879264,4.012557,32.459158,32.497482,18.077998,150,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,54.000439
4,24.041678,4.064593,34.475475,31.434920,17.984989,886,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,106.964968


In [9]:
df.describe(include='all')

,SOC (%),Voltage (V),Current (A),Battery Temp (°C),Ambient Temp (°C),Charging Cycles,Fast,Normal,Slow,Li-ion,LiFePO4,Model A,Model B,Model C,Charging Duration (min)
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,54.123090,3.854912,55.216515,29.807501,24.882106,556.560000,0.319000,0.340000,0.341000,0.497000,0.503000,0.333000,0.324000,0.343000,69.846449
std,26.292363,0.204533,26.160678,5.729787,5.736199,263.763052,0.466322,0.473946,0.474283,0.500241,0.500241,0.471522,0.468234,0.474949,28.919481
min,10.416882,3.502253,10.001047,20.013068,15.000614,101.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.618382
25%,31.237594,3.668752,33.521588,24.838015,19.899480,317.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,44.938000
50%,54.712664,3.863114,55.055253,29.685586,24.891962,571.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,69.040282
75%,76.988763,4.032326,78.319317,34.750816,29.799869,786.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,93.985178
max,99.974591,4.199590,99.803877,39.991154,34.954988,999.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,119.935030


---

<h1><center>Linear Regression Models

---

---

<h1><center>Logistic Regression Models

---

---

<h1><center>Neural Networks

---

---

<h1><center>Random Forests

---